In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep
from urllib.request import urlopen, Request
import re
import json
from IPython.display import display, clear_output
import numpy as np

### Web Scraping

In [2]:
def trata_html(html):
    html = html.decode('utf-8')
    return ' '.join(html.split()).replace('> <', '><')

def prepare_soup(url):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.100 Safari/537.36'}
    
    req = Request(url, headers = headers)
    response = urlopen(req)
    html = response.read()
    
    html = trata_html(html)
    soup = BeautifulSoup(html, 'html.parser')
    
    return soup

In [3]:
def find_links(url, type_):
    
    item_type = type_[:-2] if type_ =='Processes' else type_[:-1]
    start = 'https://www.fsaeonline.com/cdsweb/cst/Edit' + type_ + '.aspx?CatalogVersion=2132&' + item_type + 'ID='
    end = '&Action=View'
    links = []

    soup = prepare_soup(url)
    script = soup.find_all('script', type='text/javascript')[7]
    actions = re.findall("actions: '(.+?)' }", script.string)

    for action in actions:
        links.append(start + action + end)

    return links

In [4]:
def find_values(url):
    soup = prepare_soup(url)
    x = {}

    for i in soup.find_all('div', class_ = 'form-group'):
        key = i.label.get_text()
        if i.input is not None:
            value = i.input.get('value')
            
        elif i.find('option', selected="selected"):
            value = i.find('option', selected="selected").get_text()
            
        elif i.find('textarea'):
            value = i.find('textarea').text
            
        else:
            value = None
        
        if value is not None and value !='':
            x[key] = value
            x['url'] = url
    return x

In [5]:
def get_data(type_):
    start = 'https://www.fsaeonline.com/cst/Browse'
    end = '.aspx'
    
    
    url = start + type_ + end
    
    links = find_links(url,type_)
    items = []
    for link in links:
        
        item = find_values(link)
        sleep(10/1000)
        items.append(item)
        clear_output(wait=True)
        print(f'Loading {type_}... {len(items)}/{len(links)}')
    
    print('End')
    return pd.DataFrame(items)

In [6]:
# 'Fasteners', 'Materials', 'ProcessMultipliers', 'Processes', 'Tools'

fasteners_raw = get_data('Fasteners')
fasteners_raw.head()

NameError: name 'Request' is not defined

In [212]:
materials_raw = get_data('Materials')
materials_raw.head()

Loading Materials... 955/955
End


,Title*,url,Supplier,Category,Measurement Unit 1 Code,Measurement Unit 2 Code,Internal Notes,Cost Formula,Description,[C1],[C2],Size 1 Label,Size 2 Label,Common Names
0,"90-Degree Coupler, Tilton 72-560",https://www.fsaeonline.com/cdsweb/cst/EditMate...,Tilton,Brake System,[Not Used],[Not Used],These are some notes okay,45,NaN,NaN,NaN,NaN,NaN,NaN
1,"90-Degree Coupler, Tilton 72-561",https://www.fsaeonline.com/cdsweb/cst/EditMate...,Tilton,Brake System,[Not Used],[Not Used],NaN,47.5,NaN,NaN,NaN,NaN,NaN,NaN
2,"90-Degree Coupler, Tilton 72-562",https://www.fsaeonline.com/cdsweb/cst/EditMate...,Tilton,Brake System,[Not Used],[Not Used],NaN,50,NaN,NaN,NaN,NaN,NaN,NaN
3,"ABS Kit and ESP Module, Bosch, ESP-9",https://www.fsaeonline.com/cdsweb/cst/EditMate...,Bosch,Brake System,[Not Used],[Not Used],AIR from ST RQA http://www.fsaeonline.com/cds...,500,ESP-9 module only. Does not include wiring or...,NaN,NaN,NaN,NaN,NaN
4,Accusump,https://www.fsaeonline.com/cdsweb/cst/EditMate...,Canton Racing Products,Engine,L,[Not Used],NaN,([C1]*[Size1]+[C2]),"Gage not included, include as separate materi...",14.3000000000,82.0000000000,NaN,NaN,NaN


In [209]:
multipliers_raw = get_data('ProcessMultipliers')
multipliers_raw.head()

Loading ProcessMultipliers... 120/120
End


,Multiplier Type*,url,Multiplier Title,Multiplier Value,Description
0,Assembly,https://www.fsaeonline.com/cdsweb/cst/EditProc...,Assemble - Length > 0.5m,1.2500000000,NaN
1,Assembly,https://www.fsaeonline.com/cdsweb/cst/EditProc...,Disassemble,0.8000000000,To be used when removing parts or fasteners
2,Fastener Installation,https://www.fsaeonline.com/cdsweb/cst/EditProc...,Fastener Engagement Length > 2D,1.2500000000,NaN
3,Fastener Installation,https://www.fsaeonline.com/cdsweb/cst/EditProc...,Fastener Engagement Length > 4D,1.5000000000,NaN
4,"Drill, Tap",https://www.fsaeonline.com/cdsweb/cst/EditProc...,Machine - Hole Length >= 4D,1.5000000000,NaN


In [210]:
processes_raw = get_data('Processes')
processes_raw.head()

Loading Processes... 158/158
End


,Title*,url,Category,Measurement Unit 1 Code,ToolingRequired,Near Net Shape,Description,Unit Cost,Supplier,Process Multiplier Type,Internal Notes,Common Names Label,Common Names
0,Adjustment - Misc.,https://www.fsaeonline.com/cdsweb/cst/EditProc...,Labor,unit,No,No,"Chain tension, etc.",5.0000,NaN,NaN,NaN,NaN,NaN
1,Aerosol Apply,https://www.fsaeonline.com/cdsweb/cst/EditProc...,Labor,m^2,No,No,"For use with glue, adhesives, paint, etc. Inc...",5.2500,NaN,NaN,NaN,NaN,NaN
2,Annodize,https://www.fsaeonline.com/cdsweb/cst/EditProc...,Labor,unit,No,No,It is not necessary to include any cost for a...,0.0000,NaN,NaN,NaN,NaN,NaN
3,Anodizing,https://www.fsaeonline.com/cdsweb/cst/EditProc...,NaN,cm^2,No,NaN,NaN,0.0000,Any,NaN,NaN,NaN,NaN
4,"Assemble, >20 kg, Interference",https://www.fsaeonline.com/cdsweb/cst/EditProc...,Labor,unit,No,No,NaN,5.6300,NaN,Assembly,NaN,NaN,NaN


In [211]:
tools_raw = get_data('Tools')
tools_raw.head()

Loading Tools... 10/10
End


,Title*,url,Process,Measurement Unit Code,Description,Cost,Internal Notes
0,Brazing Fixture,https://www.fsaeonline.com/cdsweb/cst/EditTool...,Braze,point,Each point is a pickup or support point.,500.00,NaN
1,Die Casting - Die,https://www.fsaeonline.com/cdsweb/cst/EditTool...,Die Casting,die,Per die not die set. Minimum number of dies i...,"10,000.00",NaN
2,Lamination - Flat Panel Tool,https://www.fsaeonline.com/cdsweb/cst/EditTool...,"Lamination, Manual",m^2,Use surface area of tool that is used to form...,"1,500.00",NaN
3,Lamination - Mold Tool,https://www.fsaeonline.com/cdsweb/cst/EditTool...,"Lamination, Manual",m^2,Use surface area of tool that is used to form...,"20,000.00","Replaces 3 tools: Aluminum Tool, Composite To..."
4,Plastic injection molding - Die,https://www.fsaeonline.com/cdsweb/cst/EditTool...,Plastic injection molding,die,Per die not die set.,"10,000.00",NaN


### Saving as csv

In [213]:
fasteners_raw.to_csv('fasteners_raw.csv', index = False)
materials_raw.to_csv('materials_raw.csv', index = False)
multipliers_raw.to_csv('multipliers_raw.csv', index = False)
processes_raw.to_csv('processes_raw.csv', index = False)
tools_raw.to_csv('tools_raw.csv', index = False)

### Making prettier

In [214]:
fasteners_raw = pd.read_csv('fasteners_raw.csv')
materials_raw = pd.read_csv('materials_raw.csv')
multipliers_raw = pd.read_csv('multipliers_raw.csv')
processes_raw = pd.read_csv('processes_raw.csv')
tools_raw = pd.read_csv('tools_raw.csv')

In [215]:
dfs_raw = [fasteners_raw, materials_raw, multipliers_raw, processes_raw, tools_raw]
dfs = []
for df in dfs_raw:
    
    df = df.replace({np.nan: None, '[Not Used]': None})
    #make formulas noice
    dfs.append(df)
    
fasteners, materials, multipliers, processes, tools = dfs

In [216]:
fasteners['Label 1'] = fasteners['Size 1 Label'] + '(' + fasteners['Measurement Unit 1 Code'] + ')'
fasteners['Label 2'] = fasteners['Size 2 Label'] + '(' + fasteners['Measurement Unit 2 Code'] + ')'

map_fasteners = {'Title*' : 'Fastener', 'url' : 'Link', '[C1]' : 'C1', '[C2]': 'C2', ' Cost Formula ': 'Cost Formula'}
fasteners.rename(columns = map_fasteners, inplace = True)
fasteners = fasteners[['Fastener', 'Label 1', 'Label 2', 'Cost Formula', 'C1', 'C2', 'Description', 'Link']]

In [217]:
materials['Label 1'] = materials['Size 1 Label'] + '(' + materials['Measurement Unit 1 Code'] + ')'
materials['Label 2'] = materials['Size 2 Label'] + '(' + materials['Measurement Unit 2 Code'] + ')'

map_materials = {'Title*' : 'Material', 'url' : 'Link', '[C1]' : 'C1', '[C2]': 'C2', ' Cost Formula ': 'Cost Formula'}
materials.rename(columns = map_materials, inplace = True)

materials = materials[['Material', 'Label 1', 'Label 2', 'Cost Formula', 'C1', 'C2', 'Description', 'Supplier', 
                       'Category', 'Link']]

In [218]:
map_multipliers = {'Multiplier Type*' : 'Type', 'url' : 'Link', 'Multiplier Title': 'Multiplier'}
multipliers.rename(columns = map_multipliers, inplace = True)

multipliers = multipliers[['Multiplier', 'Type', 'Multiplier Value', 'Description', 'Link']]

In [221]:
map_processes = {'Title*' : 'Process', 'url' : 'Link', 'Measurement Unit 1 Code' : 'Unit'}
processes.rename(columns = map_processes, inplace = True)

processes = processes[['Process', 'Unit', 'Unit Cost', 'Category', 'Process Multiplier Type','ToolingRequired', 
                       'Near Net Shape', 'Description','Link']]

In [223]:
map_tools = {'Title*' : 'Tool', 'url' : 'Link', 'Measurement Unit Code' : 'Unit'}
tools.rename(columns = map_tools, inplace = True)

tools = tools[['Tool', 'Process', 'Unit', 'Cost', 'Description', 'Link']]

### Final Result

In [225]:
fasteners.head()

,Fastener,Label 1,Label 2,Cost Formula,C1,C2,Description,Link
0,Alcoa Camloc Fastener 1/4 turn,NaN,NaN,4.00,NaN,NaN,None,https://www.fsaeonline.com/cdsweb/cst/EditFast...
1,"Bolt, Aluminum",diameter(mm),length(mm),"[C1]/105154*Math.pow([Size1],2)*[Size2]*Math.s...",1.00,0.0050,Strength 255 Mpa. Special varities included (...,https://www.fsaeonline.com/cdsweb/cst/EditFast...
2,"Bolt, Grade 10.9 (SAE 8)",diameter(mm),length(mm),"[C1]/105154*Math.pow([Size1],2)*[Size2]*Math.s...",1.00,0.0040,Strength 1030 Mpa. Special varities included ...,https://www.fsaeonline.com/cdsweb/cst/EditFast...
3,"Bolt, Grade 12.9",diameter(mm),length(mm),"[C1]/105154*Math.pow([Size1],2)*[Size2]*Math.s...",1.25,0.0050,Strength 1170 Mpa. Special varities included ...,https://www.fsaeonline.com/cdsweb/cst/EditFast...
4,"Bolt, Grade 6.8 (SAE 3) and All Grades less th...",diameter(mm),length(mm),"[C1]/105154*Math.pow([Size1],2)*[Size2]*Math.s...",0.60,0.0024,Strength 670 Mpa. Special varities included (...,https://www.fsaeonline.com/cdsweb/cst/EditFast...


In [226]:
materials.head()

,Material,Label 1,Label 2,Cost Formula,C1,C2,Description,Supplier,Category,Link
0,"90-Degree Coupler, Tilton 72-560",NaN,NaN,45,NaN,NaN,None,Tilton,Brake System,https://www.fsaeonline.com/cdsweb/cst/EditMate...
1,"90-Degree Coupler, Tilton 72-561",NaN,NaN,47.5,NaN,NaN,None,Tilton,Brake System,https://www.fsaeonline.com/cdsweb/cst/EditMate...
2,"90-Degree Coupler, Tilton 72-562",NaN,NaN,50,NaN,NaN,None,Tilton,Brake System,https://www.fsaeonline.com/cdsweb/cst/EditMate...
3,"ABS Kit and ESP Module, Bosch, ESP-9",NaN,NaN,500,NaN,NaN,ESP-9 module only. Does not include wiring or...,Bosch,Brake System,https://www.fsaeonline.com/cdsweb/cst/EditMate...
4,Accusump,NaN,NaN,([C1]*[Size1]+[C2]),14.3,82.0,"Gage not included, include as separate materi...",Canton Racing Products,Engine,https://www.fsaeonline.com/cdsweb/cst/EditMate...


In [227]:
multipliers.head()

,Multiplier,Type,Multiplier Value,Description,Link
0,Assemble - Length > 0.5m,Assembly,1.25,None,https://www.fsaeonline.com/cdsweb/cst/EditProc...
1,Disassemble,Assembly,0.80,To be used when removing parts or fasteners,https://www.fsaeonline.com/cdsweb/cst/EditProc...
2,Fastener Engagement Length > 2D,Fastener Installation,1.25,None,https://www.fsaeonline.com/cdsweb/cst/EditProc...
3,Fastener Engagement Length > 4D,Fastener Installation,1.50,None,https://www.fsaeonline.com/cdsweb/cst/EditProc...
4,Machine - Hole Length >= 4D,"Drill, Tap",1.50,None,https://www.fsaeonline.com/cdsweb/cst/EditProc...


In [228]:
processes.head()

,Process,Unit,Unit Cost,Category,Process Multiplier Type,ToolingRequired,Near Net Shape,Description,Link
0,Adjustment - Misc.,unit,5.00,Labor,None,No,No,"Chain tension, etc.",https://www.fsaeonline.com/cdsweb/cst/EditProc...
1,Aerosol Apply,m^2,5.25,Labor,None,No,No,"For use with glue, adhesives, paint, etc. Inc...",https://www.fsaeonline.com/cdsweb/cst/EditProc...
2,Annodize,unit,0.00,Labor,None,No,No,It is not necessary to include any cost for a...,https://www.fsaeonline.com/cdsweb/cst/EditProc...
3,Anodizing,cm^2,0.00,None,None,No,None,None,https://www.fsaeonline.com/cdsweb/cst/EditProc...
4,"Assemble, >20 kg, Interference",unit,5.63,Labor,Assembly,No,No,None,https://www.fsaeonline.com/cdsweb/cst/EditProc...


In [229]:
tools.head()

,Tool,Process,Unit,Cost,Description,Link
0,Brazing Fixture,Braze,point,500.00,Each point is a pickup or support point.,https://www.fsaeonline.com/cdsweb/cst/EditTool...
1,Die Casting - Die,Die Casting,die,"10,000.00",Per die not die set. Minimum number of dies i...,https://www.fsaeonline.com/cdsweb/cst/EditTool...
2,Lamination - Flat Panel Tool,"Lamination, Manual",m^2,"1,500.00",Use surface area of tool that is used to form...,https://www.fsaeonline.com/cdsweb/cst/EditTool...
3,Lamination - Mold Tool,"Lamination, Manual",m^2,"20,000.00",Use surface area of tool that is used to form...,https://www.fsaeonline.com/cdsweb/cst/EditTool...
4,Plastic injection molding - Die,Plastic injection molding,die,"10,000.00",Per die not die set.,https://www.fsaeonline.com/cdsweb/cst/EditTool...


### Exporting as Excel Sheet

In [235]:
with pd.ExcelWriter('catalogs.xlsx') as writer:  
    fasteners.to_excel(writer, sheet_name='Fasteners', index = False)
    materials.to_excel(writer, sheet_name='Materials', index = False)
    multipliers.to_excel(writer, sheet_name='Multipliers', index = False)
    processes.to_excel(writer, sheet_name='Processes', index = False)
    tools.to_excel(writer, sheet_name='Tools', index = False)